<a href="https://colab.research.google.com/github/22AD081/Movie-Script-Analysis/blob/main/Movie_Script_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df = pd.read_csv("/content/imdb_movies.csv")
print(df.head())
print(df.columns)


                         names       date_x  score  \
0                    Creed III  03/02/2023    73.0   
1     Avatar: The Way of Water  12/15/2022    78.0   
2  The Super Mario Bros. Movie  04/05/2023    76.0   
3                      Mummies  01/05/2023    70.0   
4                    Supercell  03/17/2023    61.0   

                                           genre  \
0                                  Drama, Action   
1             Science Fiction, Adventure, Action   
2  Animation, Adventure, Family, Fantasy, Comedy   
3  Animation, Comedy, Family, Adventure, Fantasy   
4                                         Action   

                                            overview  \
0  After dominating the boxing world, Adonis Cree...   
1  Set more than a decade after the events of the...   
2  While working underground to fix a water main,...   
3  Through a series of unfortunate events, three ...   
4  Good-hearted teenager William always lived in ...   

                         

In [ ]:
import nltk
nltk.download('punkt_tab')
df.rename(columns={'box_office': 'box_office_label', 'genre': 'genre_label'}, inplace=True)

lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_script(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

df['clean_script'] = df['overview'].apply(preprocess_script)
print(df[['names', 'clean_script']].head())

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                         names  \
0                    Creed III   
1     Avatar: The Way of Water   
2  The Super Mario Bros. Movie   
3                      Mummies   
4                    Supercell   

                                        clean_script  
0  dominating boxing world adonis creed thriving ...  
1  set decade event first film learn story sully ...  
2  working underground fix water main brooklyn pl...  
3  series unfortunate event three mummy end prese...  
4  goodhearted teenager william always lived hope...  


In [ ]:
# Step 5: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)
script_vectors = vectorizer.fit_transform(df['clean_script'])

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

regressor = RandomForestRegressor(
    n_estimators=50,
    max_depth=15,
    n_jobs=-1,
    random_state=42
)

regressor.fit(X_train, y_train)

y_pred_score = regressor.predict(X_test)

print("Mean Squared Error:", mean_squared_error(y_test, y_pred_score))
print("R^2 Score:", r2_score(y_test, y_pred_score))


Mean Squared Error: 152.46269088343544
R^2 Score: 0.07793385924974328


In [ ]:
X_train_genre, X_test_genre, y_train_genre, y_test_genre = train_test_split(
    script_vectors,
    df['genre_label'].dropna(),
    test_size=0.2,
    random_state=42
)

genre_model = SVC(kernel='linear')
genre_model.fit(X_train_genre, y_train_genre)

y_pred_genre = genre_model.predict(X_test_genre)

print("\n=== Genre Classification Report ===")
print(classification_report(y_test_genre, y_pred_genre))
print("Confusion Matrix:\n", confusion_matrix(y_test_genre, y_pred_genre))

ValueError: Found input variables with inconsistent numbers of samples: [10178, 10093]

In [ ]:
X_train_genre, X_test_genre, y_train_genre, y_test_genre = train_test_split(
    script_vectors,
    df['genre_label'],  # Remove .dropna()
    test_size=0.2,
    random_state=42
)

# Impute or remove missing values in y_train_genre and y_test_genre
# For example, to remove rows with missing target values:

train_indices = y_train_genre.notna()
X_train_genre = X_train_genre[train_indices]
y_train_genre = y_train_genre[train_indices]

test_indices = y_test_genre.notna()
X_test_genre = X_test_genre[test_indices]
y_test_genre = y_test_genre[test_indices]

genre_model = SVC(kernel='linear')
genre_model.fit(X_train_genre, y_train_genre)

y_pred_genre = genre_model.predict(X_test_genre)

print("\n=== Genre Classification Report ===")
print(classification_report(y_test_genre, y_pred_genre))
print("Confusion Matrix:\n", confusion_matrix(y_test_genre, y_pred_genre))


=== Genre Classification Report ===
                                                                         precision    recall  f1-score   support

                                                                 Action       0.07      0.06      0.06        17
                                                      Action, Adventure       1.00      0.25      0.40         4
                                   Action, Adventure, Animation, Comedy       0.00      0.00      0.00         1
                  Action, Adventure, Animation, Family, Science Fiction       0.00      0.00      0.00         1
                                   Action, Adventure, Animation, Horror       0.00      0.00      0.00         1
                   Action, Adventure, Animation, Science Fiction, Drama       0.00      0.00      0.00         1
                                              Action, Adventure, Comedy       1.00      0.11      0.20         9
                                      Action, Adventure, C

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_